# Data Cleaning for Restaurant Analytics System

In this notebook, we will clean and preprocess two datasets:
1. **Orders.xlsx**: Contains order details.
2. **Restaurants.xlsx**: Contains restaurant inventory/menu information.

We will:
- Load the datasets
- Standardize, clean, and preprocess the data
- Merge the datasets (if necessary)
- Save the cleaned data for further analysis

In [37]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Orders Dataset

In [38]:
# Load the Orders Dataset
orders = pd.read_excel('../data/raw/Orders.xlsx')
orders.head()

,Order ID,Customer Name,Restaurant ID,Order Date,Quantity of Items,Order Amount,Payment Mode,Delivery Time Taken (mins),Customer Rating-Food,Customer Rating-Delivery
0,OD1,Srini,6,2022-01-01 23:15:00,5,633,Debit Card,47,5,3
1,OD2,Revandh,13,2022-01-01 19:21:00,5,258,Credit Card,41,3,5
2,OD3,David,9,2022-01-01 23:15:00,7,594,Cash on Delivery,30,3,4
3,OD4,Selva,4,2022-01-01 20:31:00,5,868,Cash on Delivery,30,3,4
4,OD5,Vinny,4,2022-01-01 11:10:00,4,170,Debit Card,18,4,3


In [39]:
# Inspecting the size of the excel
orders.shape

(500, 10)

In [40]:
# Inspecting columns and data types
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Order ID                    500 non-null    object        
 1   Customer Name               500 non-null    object        
 2   Restaurant ID               500 non-null    int64         
 3   Order Date                  500 non-null    datetime64[ns]
 4   Quantity of Items           500 non-null    int64         
 5   Order Amount                500 non-null    int64         
 6   Payment Mode                500 non-null    object        
 7   Delivery Time Taken (mins)  500 non-null    int64         
 8   Customer Rating-Food        500 non-null    int64         
 9   Customer Rating-Delivery    500 non-null    int64         
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 39.2+ KB


In [41]:
# Checking for missing values
orders.isnull().sum()

Order ID                      0
Customer Name                 0
Restaurant ID                 0
Order Date                    0
Quantity of Items             0
Order Amount                  0
Payment Mode                  0
Delivery Time Taken (mins)    0
Customer Rating-Food          0
Customer Rating-Delivery      0
dtype: int64

In [42]:
# Checking for duplicates
orders.duplicated().sum()

0

In [43]:
# Viewing basic statistics for numerical columns
orders.describe()

,Restaurant ID,Order Date,Quantity of Items,Order Amount,Delivery Time Taken (mins),Customer Rating-Food,Customer Rating-Delivery
count,500.000000,500,500.000000,500.000000,500.000000,500.000000,500.000000
mean,10.672000,2022-01-01 16:23:45.359999744,4.652000,598.142000,30.542000,3.362000,2.992000
min,1.000000,2022-01-01 11:10:00,1.000000,3.000000,10.000000,1.000000,1.000000
25%,5.750000,2022-01-01 13:30:00,4.000000,398.500000,21.000000,2.000000,2.000000
50%,11.000000,2022-01-01 14:31:00,5.000000,610.500000,30.000000,3.000000,3.000000
75%,16.000000,2022-01-01 20:31:00,6.000000,828.500000,41.000000,4.000000,4.000000
max,20.000000,2022-01-01 23:58:00,7.000000,1198.000000,50.000000,5.000000,5.000000
std,5.960829,NaN,1.623628,296.613981,11.883209,1.240174,1.392773


### Data Cleaning for Orders Dataset:
- Replacing the column names with lowercase and replacing the spaces with underscore.
- Extracting meaningful time-based features from order_date for further analysis.
- Converting the data type of `payment_mode` from `object` to `category`.

In [44]:
# Clean column names: lowercase, replace spaces with underscores
orders.columns = orders.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-','_')
orders.columns

Index(['order_id', 'customer_name', 'restaurant_id', 'order_date',
       'quantity_of_items', 'order_amount', 'payment_mode',
       'delivery_time_taken_(mins)', 'customer_rating_food',
       'customer_rating_delivery'],
      dtype='object')

In [45]:
# Extract meaningful time-based features from order_date
orders['day'] = orders['order_date'].dt.date
orders['hour'] = orders['order_date'].dt.hour
orders['weekday'] = orders['order_date'].dt.dayofweek
orders['month'] = orders['order_date'].dt.month

# Extracting weekends from the order_date
orders['is_weekend'] = orders['weekday'].apply(lambda x: x >= 5)

#### Standardizing the values of each column

In [46]:
# Converting customer_name and payment_mode in the same format
orders['customer_name'] = orders['customer_name'].str.strip().str.lower()
orders['payment_mode'] = orders['payment_mode'].str.strip().str.lower().str.replace(' ', '_')

# Replacing value of is_weekend to make it more analytics friendly
orders['is_weekend'] = orders['is_weekend'].replace({True: 'weekend', False: 'weekday'})

orders.head()

,order_id,customer_name,restaurant_id,order_date,quantity_of_items,order_amount,payment_mode,delivery_time_taken_(mins),customer_rating_food,customer_rating_delivery,day,hour,weekday,month,is_weekend
0,OD1,srini,6,2022-01-01 23:15:00,5,633,debit_card,47,5,3,2022-01-01,23,5,1,weekend
1,OD2,revandh,13,2022-01-01 19:21:00,5,258,credit_card,41,3,5,2022-01-01,19,5,1,weekend
2,OD3,david,9,2022-01-01 23:15:00,7,594,cash_on_delivery,30,3,4,2022-01-01,23,5,1,weekend
3,OD4,selva,4,2022-01-01 20:31:00,5,868,cash_on_delivery,30,3,4,2022-01-01,20,5,1,weekend
4,OD5,vinny,4,2022-01-01 11:10:00,4,170,debit_card,18,4,3,2022-01-01,11,5,1,weekend


In [47]:
# Converting data type from object to category
orders['payment_mode'] = orders['payment_mode'].astype('category')
orders['is_weekend'] = orders['is_weekend'].astype('category')

In [48]:
orders.dtypes

order_id                              object
customer_name                         object
restaurant_id                          int64
order_date                    datetime64[ns]
quantity_of_items                      int64
order_amount                           int64
payment_mode                        category
delivery_time_taken_(mins)             int64
customer_rating_food                   int64
customer_rating_delivery               int64
day                                   object
hour                                   int32
weekday                                int32
month                                  int32
is_weekend                          category
dtype: object

In [49]:
orders.describe()

,restaurant_id,order_date,quantity_of_items,order_amount,delivery_time_taken_(mins),customer_rating_food,customer_rating_delivery,hour,weekday,month
count,500.000000,500,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.0,500.0
mean,10.672000,2022-01-01 16:23:45.359999744,4.652000,598.142000,30.542000,3.362000,2.992000,16.00000,5.0,1.0
min,1.000000,2022-01-01 11:10:00,1.000000,3.000000,10.000000,1.000000,1.000000,11.00000,5.0,1.0
25%,5.750000,2022-01-01 13:30:00,4.000000,398.500000,21.000000,2.000000,2.000000,13.00000,5.0,1.0
50%,11.000000,2022-01-01 14:31:00,5.000000,610.500000,30.000000,3.000000,3.000000,14.00000,5.0,1.0
75%,16.000000,2022-01-01 20:31:00,6.000000,828.500000,41.000000,4.000000,4.000000,20.00000,5.0,1.0
max,20.000000,2022-01-01 23:58:00,7.000000,1198.000000,50.000000,5.000000,5.000000,23.00000,5.0,1.0
std,5.960829,NaN,1.623628,296.613981,11.883209,1.240174,1.392773,4.03144,0.0,0.0


- Certain anamoly here as the __order_amount__'s minimum value is __$3__ which can be a possible business error
- No outliers as such are found here

In [50]:
# Storing the cleaned orders data
orders.to_pickle('../data/cleaned/orders_cleaned.pkl')

## Restaurants Dataset

In [51]:
# Load the Restaurants Dataset
restaurants = pd.read_excel('../data/raw/Restaurants.xlsx')
restaurants.head()

,RestaurantID,RestaurantName,Cuisine,Zone,Category
0,1,The Cave Hotel,Continental,Zone B,Pro
1,2,SSK Hotel,North Indian,Zone D,Pro
2,3,ASR Restaurant,South Indian,Zone D,Ordinary
3,4,Win Hotel,South Indian,Zone D,Ordinary
4,5,Denver Restaurant,Continental,Zone D,Pro


In [52]:
# Inspecting the size of the excel
restaurants.shape

(20, 5)

In [53]:
# Inspecting columns and data types
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RestaurantID    20 non-null     int64 
 1   RestaurantName  20 non-null     object
 2   Cuisine         20 non-null     object
 3   Zone            20 non-null     object
 4   Category        20 non-null     object
dtypes: int64(1), object(4)
memory usage: 932.0+ bytes


In [54]:
# Checking for missing values
restaurants.isnull().sum()

RestaurantID      0
RestaurantName    0
Cuisine           0
Zone              0
Category          0
dtype: int64

In [55]:
# Checking for duplicates values
restaurants.duplicated().sum()

0

In [56]:
print(restaurants['RestaurantID'].value_counts().sum())
print(restaurants['Category'].unique())
print(restaurants['Zone'].unique())
print(restaurants['Cuisine'].unique())

20
['Pro' 'Ordinary']
['Zone B' 'Zone D' 'Zone A' 'Zone C']
['Continental' 'North Indian' 'South Indian' 'French' 'Chinese' 'African'
 'Arabian' 'Belgian']


- The __restaurantID__ being 20 shows us that all the restaurant are unique since the shape says that 20 rows are present.
- Thus we can see from here that `cuisine`, `zone` and `category` can be marked as data type of `category` only.

### Data Cleaning for Restaurants Dataset:
- Replacing the column names with lowercase and adding underscore wherever required.
- Standardizing the data
- Converting the data type of `cuisine`, `zone` and `category` from `object` to `category`.
- Converting the data type of `ResturantName` from `object` to `string`.

In [57]:
restaurants.rename(columns={
    'RestaurantID': 'restaurant_id',
    'RestaurantName': 'restaurant_name',
    'Cuisine': 'cuisine',
    'Zone': 'zone',
    'Category': 'category'
}, inplace=True)
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'zone', 'category'], dtype='object')

In [58]:
# Standardizing the data in a format
restaurants = restaurants.apply(lambda col: col.str.strip().str.lower().str.replace(' ', '_') if col.dtypes == 'object' else col)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,zone,category
0,1,the_cave_hotel,continental,zone_b,pro
1,2,ssk_hotel,north_indian,zone_d,pro
2,3,asr_restaurant,south_indian,zone_d,ordinary
3,4,win_hotel,south_indian,zone_d,ordinary
4,5,denver_restaurant,continental,zone_d,pro


In [59]:
restaurants[['cuisine', 'zone', 'category']] = restaurants[['cuisine', 'zone', 'category']].astype('category')
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   restaurant_id    20 non-null     int64   
 1   restaurant_name  20 non-null     object  
 2   cuisine          20 non-null     category
 3   zone             20 non-null     category
 4   category         20 non-null     category
dtypes: category(3), int64(1), object(1)
memory usage: 1.2+ KB


In [60]:
restaurants.describe()

,restaurant_id
count,20.00000
mean,10.50000
std,5.91608
min,1.00000
25%,5.75000
50%,10.50000
75%,15.25000
max,20.00000


- No __outliers__ as such are found here

In [61]:
# Storing the cleaned restaurants data
restaurants.to_pickle('../data/cleaned/restaurants_cleaned.pkl')